In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import the libararies

In [ ]:
from tqdm import tqdm
import os
from random import randint

import numpy as np
import pandas as pd

import nibabel as nib
import pydicom as pdm
import nilearn as nl
import nilearn.plotting as nlplt
#import nrrd
import h5py
import pydicom

from glob import glob

import imageio
from IPython.display import Image

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim

import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif
import numpy.ma as ma
import warnings
warnings.simplefilter("ignore")

In [ ]:
# to display dicom file 
path = r"../input/osic-pulmonary-fibrosis-progression/train"
series = np.array([[(os.path.join(dp, f), pydicom.dcmread(os.path.join(dp, f), stop_before_pixels = True)) for f in files]
                   for dp,_,files in os.walk(path) if len(files) != 0])

# to make list of dicom files 
PATH='../input/osic-pulmonary-fibrosis-progression/train'
## First, read all of my DICOM files into a list
mydicoms = glob("../input/osic-pulmonary-fibrosis-progression/train/*/*.dcm")

# to show the meta_Data of any patient 

In [ ]:
mydicoms [0][3][1]


In [ ]:
def specific_dicom_display(path=path):
    dcm=pydicom.dcmread(path)
    pat_name = dcm.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name................:", display_name)
    print("Patient id....................:", RefDs.PatientID)
    print("Modality......................:", RefDs.Modality)
    print("BodyPartExamined..............:", RefDs.BodyPartExamined)  
    print("Image Position    (Patient)...:", RefDs.ImagePositionPatient)
    print("Image Orientation (Patient)...:", RefDs.ImageOrientationPatient)

In [ ]:
specific_dicom_display('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/1.dcm')

In [ ]:
instances = [f for l in series for f in l]
print(len(instances))
patient_ids = np.unique([inst[1].PatientID for inst in instances])
print("the number of patients :",len(patient_ids))
#-------------------------------------------------------------------------------
print("Number of DICOM files:", len(os.listdir("../input/osic-pulmonary-fibrosis-progression/train")))

#---------------------------------------------------------------

print("the number of series :",len(series))


#-----------------------------------------------------------
# How many studies?

studies = {}

for s in series:
    studies.setdefault(s[0][1].StudyInstanceUID, []).append(s)
print("the number of studies :",len(studies))

#---------------------------------------------------
[len([st for st in studies.values() if st[0][0][1].PatientID == p]) for p in patient_ids]



#-------------------------------------------------
# Let's see how many series per study

series_per_study = [(len(sr), sr[0][0][1].PatientID) for sr in studies.values()]
#print("the number of series per study :",series_per_study)


#-----------------------------------------------------
img_per_series = [len(s) for s in series]

print("the number of images per series :",img_per_series)

# the outliers for number of images 
**3 for studies more than 600 image

**4 for studies more_500_less_600 image

In [ ]:
plt.hist(img_per_series)
plt.show()
more_600=[]
less_100=[]
more_100_less_200=[]
more_200_less_300=[]
more_300_less_400=[]
more_400_less_500=[]
more_500_less_600=[]
for i in range(len(img_per_series)):
    
    if img_per_series[i]<100:
        less_100.append(img_per_series[i])
     
    elif((img_per_series[i]>=100) and (img_per_series[i]<200)):
        more_100_less_200.append(img_per_series[i])
        
    elif((img_per_series[i]>=200) and (img_per_series[i]<300)):
        more_200_less_300.append(img_per_series[i])
        
    elif((img_per_series[i]>=300) and (img_per_series[i]<400)):
        more_300_less_400.append(img_per_series[i])
        
    elif((img_per_series[i]>=400) and (img_per_series[i]<500)):
        more_400_less_500.append(img_per_series[i])
                                 
    elif((img_per_series[i]>=500) and (img_per_series[i]<600)):
        more_500_less_600.append(img_per_series[i])
                                 
    else:
        more_600.append(img_per_series[i])
                                 
                                 
print("the length of studies more than 600 image is:{} ",len(more_600))
print("the length of studies less than 100 image is:{} ",len(less_100))
print("the length of studies more_100_less_200 image is:{} ",len(more_100_less_200))                                 
print("the length of studies more_200_less_300 image is:{} ",len(more_200_less_300))                                 
print("the length of studies more_300_less_400 image is:{} ",len(more_300_less_400))
print("the length of studies more_400_less_500 image is:{} ",len(more_400_less_500))
print("the length of studies more_500_less_600 image is:{} ",len(more_500_less_600))

In [ ]:
def process_data(path):
    data = pd.DataFrame([{'path': filepath} for filepath in glob(PATH+path)])
    data['file'] = data['path'].map(os.path.basename)
    for i in mydicoms:
        dicom_read = pydicom.dcmread(i)
    data['ID'] = dicom_read.PatientID
    #data['Age'] = dicom_read.PatientAge
    data['Modality'] = dicom_read.Modality
    #data['description']=dicom_read.StudyDescription
    return data

In [ ]:
dicom_data = process_data('/*/*.dcm')


In [ ]:
dicom_data.shape[0]

In [ ]:
dim = dicom_data.shape[0]
size = 6
data2Dlist = [[0 for x in range(size)] for y in range(dim)] 
lstFilesDCM =mydicoms
for i in range(0,dim):
    #print(i, lstFilesDCM[i])
    data2Dlist[i][0] = pydicom.dcmread(lstFilesDCM[i]).PatientID
    data2Dlist[i][1] = pydicom.dcmread(lstFilesDCM[i]).Modality
    data2Dlist[i][2] = pydicom.dcmread(lstFilesDCM[i]).BodyPartExamined
    data2Dlist[i][3] = pydicom.dcmread(lstFilesDCM[i]).InstanceNumber
    data2Dlist[i][4] = '73'
    data2Dlist[i][5] = 'male'
                      

data2Dlist
df = pd.DataFrame(data2Dlist, columns=['ID', 'Modality', 'BPE', 'Slice', 'Age', 'Sex'])
print(df.head(10))

In [ ]:
data=np.concatenate([df,dicom_data],axis=1)

data=pd.DataFrame(data,columns=['ID', 'Modality', 'BPE', 'Slice', 'Age', 'Sex','DIRECTORY','FILE','I','M'])

print(df.shape[0],dicom_data.shape[0])


In [ ]:
data.head(10)


In [ ]:
data=data.drop(['I'], axis = 1) 
data=data.drop(['M'], axis = 1) 

In [ ]:
data.head(10)

In [ ]:
data_train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
data_test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
data_train.head(10)

In [ ]:
def subplotting(column):
    fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
    axs[0].hist(data_train[column] )
    plt.title("TRAIN")
    axs[1].hist(data_test[column])
    plt.title("TEST")
    plt.suptitle("the train vs test for %s "%column)
    plt.show()

In [ ]:
subplotting("Weeks")
subplotting("FVC")
subplotting("Percent")
subplotting("Age")
subplotting("Sex")
subplotting("SmokingStatus")



In [ ]:
import pydicom
from pydicom.data import get_testdata_files
import numpy as np

print(__doc__)

PathDicom = '/kaggle/input/osic-pulmonary-fibrosis-progression/'
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))


fig, axs = plt.subplots(4,4, figsize=(25,25))
for i in range(0,16):
    RefDs = pydicom.dcmread(lstFilesDCM[i])
    axs[i//4, i%4].imshow(RefDs.pixel_array, cmap='gray') 
    axs[i//4, i%4].margins(x=0.5, y=-0.25)   # Values in (-0.5, 0.0) zooms in to center
    axs[i//4, i%4].set_axis_off()
    axs[i//4, i%4].set_title('Modality: {} BPE: {}\n Slice: {} Age: {} Sex: {}'.format(df.Modality[i],df.BPE[i],df.Slice[i],df.Age[i],df.Sex[i]))

plt.axis("off") 
plt.show()

the visulaization of dicom file 


In [ ]:
def load_scan(path):
    """
    Loads scans from a folder and into a list.
    
    Parameters: path (Folder path)
    
    Returns: slices (List of slices)
    """
    
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices
def get_pixels_hu(scans):
    """
    Converts raw images to Hounsfield Units (HU).
    
    Parameters: scans (Raw images)
    
    Returns: image (NumPy array)
    """
    
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)

    # Since the scanning equipment is cylindrical in nature and image output is square,
    # we set the out-of-scan pixels to 0
    image[image == -2000] = 0
    
    
    # HU = m*P + b
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)
def set_lungwin(img, hu=[-1200., 600.]):
    lungwin = np.array(hu)
    newimg = (img-lungwin[0]) / (lungwin[1]-lungwin[0])
    newimg[newimg < 0] = 0
    newimg[newimg > 1] = 1
    newimg = (newimg * 255).astype('uint8')
    return newimg




In [ ]:
scans = load_scan('../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430/')
scan_array = set_lungwin(get_pixels_hu(scans))

imageio.mimsave("/tmp/gif.gif", scan_array, duration=0.0001)
Image(filename="/tmp/gif.gif", format='png')